<a href="https://colab.research.google.com/github/ParmarKrishna/native-language-identification/blob/main/native_language_identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Native Language Identification
>  Author: Krishna Parmar



## Connecting google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Importing neccessary libraries

In [ ]:
!pip install librosa
!pip install numpy
!pip install scipy
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import librosa
import numpy as np
import pandas as pd
import os
import subprocess
from tqdm import tqdm

## Base Variables

In [ ]:
MAIN_DIR_PATH = '/content/drive/Shareddrives/speech_communication'
MAIN_DB_PATH = '/content/drive/Shareddrives/speech_database'

In [ ]:
%cd {MAIN_DIR_PATH}

/content/drive/Shareddrives/speech_communication


## Utilities

In [ ]:
def prRed(skk): print("\033[91m {}\033[00m" .format(skk))
 
 
def prGreen(skk): print("\033[92m {}\033[00m" .format(skk))
 
 
def prYellow(skk): print("\033[93m {}\033[00m" .format(skk))
 
 
def prLightPurple(skk): print("\033[94m {}\033[00m" .format(skk))
 
 
def prPurple(skk): print("\033[95m {}\033[00m" .format(skk))
 
 
def prCyan(skk): print("\033[96m {}\033[00m" .format(skk))
 
 
def prLightGray(skk): print("\033[97m {}\033[00m" .format(skk))
 
 
def prBlack(skk): print("\033[98m {}\033[00m" .format(skk))

In [ ]:
os.chdir(MAIN_DB_PATH)

## MFCC coefficients and Data loader
> CNN attempt

### Loading folders

In [ ]:
folders = os.listdir()

In [ ]:
len(folders)

7

In [ ]:
folders

['.ipynb_checkpoints',
 'scripts',
 'Hindi',
 'Kannada',
 'Malayalam',
 'Tamil',
 'Telugu']

In [ ]:
ignore = ['scripts','.ipynb_checkpoints']

In [ ]:
for folder in folders:
  if folder in ignore:
    folders.remove(folder)

folders

['Hindi', 'Kannada', 'Malayalam', 'Tamil', 'Telugu']

### Indexing

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/Shareddrives/speech_database/indexs.csv')

In [ ]:
df.head()

,FILEPATH,LANGUAGE
0,/content/drive/Shareddrives/speech_database/Hi...,0
1,/content/drive/Shareddrives/speech_database/Hi...,0
2,/content/drive/Shareddrives/speech_database/Hi...,0
3,/content/drive/Shareddrives/speech_database/Hi...,0
4,/content/drive/Shareddrives/speech_database/Hi...,0


In [ ]:
df.loc[0,'FILEPATH']

'/content/drive/Shareddrives/speech_database/Hindi/Hindi/RECS/0025/Hin_0025_Hin_m_0001.wav'

In [ ]:
def spec(filepath):
  y,sr=librosa.load(filepath,sr=44100)
  D = np.abs(librosa.stft(y))**2
  S =librosa.feature.melspectrogram(y=y, sr=sr, n_mels=20,fmax=8000)
  p_S = S
  s = S.shape
  S.resize(1,s[0],s[1])
  return S,p_S

In [ ]:
import librosa.display

In [ ]:
def pad_trunc(aud, max_ms):
  audio, sr = aud
  num_rows, sig_len = audio.shape
  max_len = sr//1000 * max_ms
  if (sig_len > max_len):
    audio = audio[:,:max_len]

  elif (sig_len < max_len):
    pad_begin_len = random.randint(0, max_len - sig_len)
    pad_end_len = max_len - sig_len - pad_begin_len

      # Pad with 0s
    pad_begin = torch.zeros((num_rows, pad_begin_len))
    pad_end = torch.zeros((num_rows, pad_end_len))

    audio = torch.cat((pad_begin, audio, pad_end), 1)
      
  return (audio, sr)

In [ ]:
import os
import pandas as pd
from torch.utils.data import Dataset,random_split
class speech_dataset(Dataset):
    def __init__(self,df,data_path):
      self.df=df
      self.sr=44100

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        filepath = self.df.loc[idx,'FILEPATH']
        lang = self.df.loc[idx,'LANGUAGE']
        sgram = spec(filepath)
        sgram = pad_trunc(sgram,7000)
        return sgram

In [ ]:
from sklearn.utils import shuffle
shuffle(df)

,FILEPATH,LANGUAGE
1029,/content/drive/Shareddrives/speech_database/Hi...,0
9169,/content/drive/Shareddrives/speech_database/Ta...,3
11038,/content/drive/Shareddrives/speech_database/Ta...,3
4525,/content/drive/Shareddrives/speech_database/Ka...,1
2069,/content/drive/Shareddrives/speech_database/Hi...,0
...,...,...
6458,/content/drive/Shareddrives/speech_database/Ka...,1
5904,/content/drive/Shareddrives/speech_database/Ka...,1
12407,/content/drive/Shareddrives/speech_database/Te...,4
924,/content/drive/Shareddrives/speech_database/Hi...,0


### CNN

In [ ]:
# import torch.utils.data import random_split
data = speech_dataset(df,'/content/drive/Shareddrives/speech_database/Hindi/Hindi/RECS')
train_set = round(len(data)*0.8)
test_set = len(data)-train_set
train_set,test_set = random_split(data,[train_set,test_set])

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=32, shuffle=True)

In [ ]:
result=0
accuracy=0

In [ ]:
train_dataloader.dataset.dataset

,FILEPATH,LANGUAGE
0,/content/drive/Shareddrives/speech_database/Hi...,0
1,/content/drive/Shareddrives/speech_database/Hi...,0
2,/content/drive/Shareddrives/speech_database/Hi...,0
3,/content/drive/Shareddrives/speech_database/Hi...,0
4,/content/drive/Shareddrives/speech_database/Hi...,0
...,...,...
13573,/content/drive/Shareddrives/speech_database/Te...,4
13574,/content/drive/Shareddrives/speech_database/Te...,4
13575,/content/drive/Shareddrives/speech_database/Te...,4
13576,/content/drive/Shareddrives/speech_database/Te...,4


In [ ]:
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
import torch.nn.functional as F
from torch.nn import init
import torch.nn as nn
class LanguageClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        conv_layers = []

        self.conv1 = nn.Conv2d(2, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
        self.relu1 = nn.ReLU()
        self.bn1 = nn.BatchNorm2d(8)
        init.kaiming_normal_(self.conv1.weight, a=0.1)
        self.conv1.bias.data.zero_()
        conv_layers += [self.conv1, self.relu1, self.bn1]

        self.conv2 = nn.Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu2 = nn.ReLU()
        self.bn2 = nn.BatchNorm2d(16)
        init.kaiming_normal_(self.conv2.weight, a=0.1)
        self.conv2.bias.data.zero_()
        conv_layers += [self.conv2, self.relu2, self.bn2]

        self.conv3 = nn.Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu3 = nn.ReLU()
        self.bn3 = nn.BatchNorm2d(32)
        init.kaiming_normal_(self.conv3.weight, a=0.1)
        self.conv3.bias.data.zero_()
        conv_layers += [self.conv3, self.relu3, self.bn3]

        self.conv4 = nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu4 = nn.ReLU()
        self.bn4 = nn.BatchNorm2d(64)
        init.kaiming_normal_(self.conv4.weight, a=0.1)
        self.conv4.bias.data.zero_()
        conv_layers += [self.conv4, self.relu4, self.bn4]

        self.ap = nn.AdaptiveAvgPool2d(output_size=1)
        self.lin = nn.Linear(in_features=64, out_features=5)

        self.conv = nn.Sequential(*conv_layers)

    def forward(self, x):
        x = self.conv(x)
        x = self.ap(x)
        x = x.view(x.shape[0], -1)
        x = self.lin(x)
        return x

myModel =LanguageClassifier()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
myModel = myModel.to(device)
next(myModel.parameters()).device

device(type='cpu')

In [ ]:
myModel.eval()

LanguageClassifier(
  (conv1): Conv2d(2, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (relu1): ReLU()
  (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (relu2): ReLU()
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (relu3): ReLU()
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (relu4): ReLU()
  (bn4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (ap): AdaptiveAvgPool2d(output_size=1)
  (lin): Linear(in_features=64, out_features=5, bias=True)
  (conv): Sequential(
    (0): Conv2d(2, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (1): ReLU()
    (2): BatchNorm2d(8, eps=1e-05, mo

In [ ]:
def training(model, train_set, num_epochs):
  # Loss Function, Optimizer and Scheduler
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
  scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001,
                                                steps_per_epoch=int(len(train_set)),
                                                epochs=num_epochs,
                                                anneal_strategy='linear')

  # Repeat for each epoch
  for epoch in range(num_epochs):
    running_loss = 0.0
    correct_prediction = 0
    total_prediction = 0

    # Repeat for each batch in the training set
    for i, data in enumerate(train_set):
        # Get the input features and target labels, and put them on the GPU
        inputs, labels = data[0].to(device), data[1].to(device)

        # Normalize the inputs
        inputs_m, inputs_s = inputs.mean(), inputs.std()
        inputs = (inputs - inputs_m) / inputs_s

        # Zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

        # Keep stats for Loss and Accuracy
        running_loss += loss.item()
        _, prediction = torch.max(outputs,1)
        correct_prediction += (prediction == labels).sum().item()
        total_prediction += prediction.shape[0]
    num_batches = len(train_set)
    avg_loss = running_loss / num_batches
    acc = correct_prediction/total_prediction
    print(f'Epoch: {epoch}, Loss: {avg_loss:.2f}, Accuracy: {acc:.2f}')

  print('Finished Training')
  
num_epochs=10   
training(myModel, train_dataloader, num_epochs)

In [ ]:
for i, batch in enumerate(train_dataloader.dataset):
    print(i, batch)

In [ ]:
%cd {MAIN_DIR_PATH}

/content/drive/Shareddrives/speech_communication
